<a href="https://colab.research.google.com/github/csciulla/stress-test-dashboard/blob/main/stresstest_ntbk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize
from hmmlearn.hmm import GaussianHMM

In [ ]:
class Portfolio:
  def __init__(self, portfolio:list,  lower_bound:float, upper_bound:float):
    try:
      if lower_bound >= upper_bound:
        raise ValueError("Lower bound must be less than upper bound.")

      self.portfolio = portfolio
      self.weights = None
      self.dfclose = None
      self.lower_bound = lower_bound
      self.upper_bound = upper_bound

    except Exception as e:
      print(f"Error in intializer function: {e}")
      return None

  def get_data(self, period:str=None, start_date:str=None, end_date:str=None):
    """
    Returns a dataframe of the historical adjusted close prices of the assets.
    - Only one method of date input should be provided, either 'period' or 'start_date' and 'end_date'.
    - Length of time series should be large enough to handle metric calculations.

    Parameters:
    - period: yfinance time period (e.g., '3mo', '1y', '3y', '5y', 'ytd', 'max').
    - start_date: Start date of the time series. YYYY-MM-DD format.
    - end_date: End date of the time series. YYYY-MM-DD format.
    """
    try:
      if period and (start_date or end_date): #checks if both methods of date input are used
        raise ValueError("Provide either 'period' OR both 'start_date' and 'end_date' -- not both.")

      if period:
        period = period.strip()
        self.dfclose = yf.download(self.portfolio, period=period, progress=False, auto_adjust=False)["Adj Close"]
      elif start_date and end_date:
        start_date = start_date.strip()
        end_date = end_date.strip()
        self.dfclose = yf.download(self.portfolio, start=start_date, end=end_date, progress=False, auto_adjust=False)["Adj Close"]
      else:
        raise ValueError("You must provide either a 'period' or both 'start_date' and 'end_date'.")

      if self.dfclose.empty or self.dfclose is None:
        raise ValueError("Downloaded price data is empty or unavailable.")
      elif len(self.dfclose) <= 2:
        raise ValueError("Downloaded price data is too short.")
      elif len(self.dfclose) < 21: #average trading days in a month
        print("Warning: Limited price history may lead to unreliable metrics.")

      return self.dfclose

    except Exception as e:
      print(f"Error in get_data: {e}")
      return None

  def get_weights(self, type_weight:str):
    """
    Returns a list of weights for the portfolio.

    Parameters:
    - type_weight: Input 'eq' for equal-weighted portfolio or 'opt' for optimized weights based on the Sharpe-Ratio
    """
    try:
      dfclose = self.dfclose
      if dfclose is None or dfclose.empty:
        raise ValueError("The portfolio's price data is missing. Please properly run 'get_data' first.")
      elif len(dfclose) <= 2:
        raise ValueError("Downloaded price data is too short.")

      #Get log returns of each asset
      log_returns = np.log(dfclose/dfclose.shift()).dropna()

      #Calculate initial portfolio metrics
      weights = np.repeat(1/len(self.portfolio), len(self.portfolio))
      expected_returns = log_returns.mean()*252
      port_returns = weights.T @ expected_returns
      cov_matrix = log_returns.cov()*252
      port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
      rf = 0.045

      #Set bounds and constraints for objective function
      bounds = [(self.lower_bound, self.upper_bound) for _ in range(len(self.portfolio))]
      constraints = {"type": "eq", "fun": lambda w: np.sum(w)-1}
      def neg_sharpe(w):
        port_ret = w.T @ expected_returns
        port_std = np.sqrt(w.T @ cov_matrix @ w)
        return -((port_ret - rf)/port_std)

      if type_weight.strip().lower() == "eq":
        self.weights = [float(i) for i in weights]
      elif type_weight.strip().lower() == "opt":
        optimized_weights = minimize(neg_sharpe, weights, method="SLSQP", bounds=bounds, constraints=constraints)
        self.weights = [round(float(i),4) for i in optimized_weights.x]
      else:
        raise ValueError("Select a valid input for 'type_weight' -- either 'eq' or 'opt'.")

      return self.weights

    except Exception as e:
      print(f"Error in get_weights: {e}")
      return None


In [147]:
#test weights
test = Portfolio(["AAPL", "MSFT", "GOOG", "JNJ", "XOM"], 0.0, 0.5)
test.get_data('10y')
test.get_weights("opt")

[0.3106, 0.0766, 0.1128, 0.5, 0.0]

In [ ]:
def monte_carlo(T:int, sims:int, weights:list, df:pd.DataFrame, regime:str, level:str, factorReturns:list=None, rand:bool=None ):
  """
  Returns simulated portfolio returns using Monte Carlo Simulation.

  Parameters:
  - T: Number of days in each simulation.
  - sims: Number of simulations.
  - weights: List of asset weights.
  - df: Dataframe of the historical adjusted close prices of the assets.
  - regime: Determines how much or how little the portfolio is affected by the crisis event.   
  - level: Scale of the crisis event.
  - 
  - rand: input the boolean 'True' to return a random simulation, otherwise ignore

    regime options: 'Low', 'Medium', 'High'
    level options: 'Mild', 'Moderate', 'Severe', 'Tail Risk', 'Regulatory'
  """
  try:
    if T <= 2:
      raise ValueError("The length of each simulated path is too short.")
    elif T < 21:
      print("Warning: Limited price data may lead to unreliable metrics.")

    #Intialize dictionary to store simulated paths of T days for each ticker
    tickers = list(df.columns) + ['SPY']
    weights = weights + [0.000]
    sims_returns = {ticker: np.full(shape=(T, sims), fill_value=0.0) for ticker in tickers}
    
    #Correspond regime with scaling factor
    regime = regime.strip().capitalize()
    level = level.strip().capitalize()
    factorDict = {"Mild": 1.0,
                  "Moderate": 1.3,
                  "Severe": 1.7,
                  "Tail risk": 2.0,
                  "Regulatory": 2.5}
    scaling_factor = factorDict[level]

    #Calculate log returns and align with market returns
    start_date = pd.to_datetime(df.index[0])
    end_date = pd.to_datetime(df.index[-1])
    market = yf.download('SPY', start=start_date, end=end_date, progress=False, auto_adjust=False)['Adj Close']
    market_returns = np.log(market/market.shift()).dropna()
    log_returns = np.log(df/df.shift()).dropna()
    aligned_index = log_returns.index.intersection(market_returns.index)
    market_returns = market_returns.loc[aligned_index]
    log_returns = log_returns.loc[aligned_index]
    log_returns['SPY'] = market_returns

    #Create mean matrix 
    if factorReturns is not None:
      meanM = np.full(shape=(T, len(tickers)), fill_value=factorReturns)
    else:
      expected_return = log_returns.mean()
      meanM = np.full(shape=(T, len(tickers)), fill_value=expected_return)

    #Initalize HMM
    port_returns = (log_returns @ weights).values.reshape(-1,1) #HMM requires 2D array
    historical_port_vol = np.std(port_returns)
    model = GaussianHMM(n_components=3, covariance_type="full", n_iter=1000, random_state=42)
    model.fit(port_returns)

    #Gather the volatility regimes established by the HMM and correspond them with their respective state
    vol_states = ["Low","Medium","High"]
    vol_regimes = np.sqrt([var[0][0] for var in model.covars_])
    vol_regimes = np.sort(vol_regimes)
    vol_dict = {state: vol for state, vol in zip(vol_states, vol_regimes)}

    #Calculate the scale factor needed for the historical data to reach the desired volatility and then apply it to L
    desired_vol = vol_dict[regime]*scaling_factor
    vol_scale_factor = desired_vol / historical_port_vol
    cov_matrix = log_returns.cov()* (vol_scale_factor**2)
    L = np.linalg.cholesky(cov_matrix)

    #Generate paths
    for m in range(sims):
      Z = np.random.normal(size=(T, len(tickers)))
      dailyReturns = meanM + Z @ L.T
      for i, ticker in enumerate(tickers):
        sims_returns[ticker][:,m] = dailyReturns[:,i]

    #Get a random path
    if rand:
      random_int = np.random.randint(0,sims)
      random_sims_returns = {ticker: sims_returns[ticker][:,random_int] for ticker in tickers}
      random_sims_df = pd.DataFrame(random_sims_returns)
      return random_sims_df
    elif rand != None:
      raise ValueError("Invaild input for 'rand'. Input the string 'yes' to return a random path, otherwise ignore.")
    else:
        return sims_returns

  except Exception as e:
    print(f"Error in monte_carlo: {e}")
    return None

In [63]:
df = test.get_data('5y')
monte_carlo(T=25, sims=5, weights=[0.0, 0.0596, 0.0, 0.4404, 0.5], df=df, regime="Low", level='Mild', rand=True)

,AAPL,GOOG,JNJ,MSFT,XOM,SPY
0,-0.011233,0.007702,0.003213,0.007368,-0.008796,-0.003690
1,-0.023796,-0.018434,0.004512,-0.020335,0.000869,-0.007726
2,-0.020312,-0.027898,0.015285,-0.026217,-0.007628,-0.013072
3,-0.003836,0.007549,-0.001014,-0.001690,-0.035535,-0.007008
4,0.007277,-0.004699,0.001178,-0.001735,0.002038,-0.000686
5,0.006275,0.009345,-0.007587,0.000531,0.005600,0.001013
6,0.002775,-0.004645,-0.008180,-0.007393,-0.012840,-0.003766
7,-0.000815,0.018523,-0.006021,-0.013282,0.015894,-0.003937
8,0.030708,0.010986,0.001723,0.036041,-0.004538,0.018695
9,0.028279,0.033311,-0.004992,0.026799,-0.002451,0.013532


In [ ]:
def calculate_metrics(weights:list, df:pd.DataFrame):
  """
  Calculates the metrics and the percent contribution of risk for each stock in the portfolio.
  - Metrics returned: Annual portfolio volatilty, Sharpe Ratio, 95% VaR, 95% CVaR, Max Drawdown, and Beta.

  Parameters:
  - weights: List of each assets weight in the portfolio
  - df: Dataframe of the historical adjusted close prices of the assets or simulated returns via the monte_carlo function
  """
  try:
    if df is None or df.empty:
      raise ValueError("Price data is empty or unavailable. Make sure historical/simulated data is properly downloaded.")

    #Core calculations
    if df.iloc[0,0] < 1:
      log_returns = df
      weights = weights + [0.000]
    else:
      log_returns = np.log(df/df.shift()).dropna()

    tickers = list(df.columns)
    weights = np.array(weights)
    expected_returns = log_returns.mean()*252
    cov_matrix = log_returns.cov()*252
    rf = 0.045
    port_returns = weights.T @ expected_returns
    port_returns_series = log_returns @ weights

    #Metrics
    port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    sharpe = (port_returns - rf)/port_vol
    VaR_95 = np.percentile(port_returns_series, 5)
    CVaR_95 = port_returns_series[port_returns_series <= VaR_95].mean()

    #Max Drawdown
    cum_returns = (1+port_returns_series).cumprod()
    cum_max = np.maximum.accumulate(cum_returns)
    drawdown = cum_returns/cum_max - 1
    mdd = drawdown.min() #drawdown values are negative

    #Beta
    if pd.api.types.is_integer_dtype(port_returns_series.index):
      #Simulated case: align by length
      market_returns = df['SPY']
    else:
      market = yf.download("SPY", period='max', progress=False, auto_adjust=False)["Adj Close"]
      market_returns = (np.log(market/market.shift()).dropna()).squeeze() #convert to series so that it works properly with port_returns_series
      
      #Simulated historical case: align by date
      start_date = pd.to_datetime(port_returns_series.index[0])
      end_date = pd.to_datetime(port_returns_series.index[-1])
      if start_date and end_date not in market_returns.index: #first make sure that market data contains crisis event
        market = yf.download("SPY", start=start_date, end=end_date, progress=False, auto_adjust=False)["Adj Close"]
        market_returns = (np.log(market/market.shift()).dropna()).squeeze()

      #align by date for either simulated historical or historical case
      aligned_index = port_returns_series.index.intersection(market_returns.index)
      market_returns = market_returns.loc[aligned_index]
      port_returns_series = port_returns_series.loc[aligned_index]
    beta = port_returns_series.cov(market_returns) / market_returns.var()

    #Calculate PCR (Percent Contribution to Risk)
    PCRdict = {}
    if 'SPY' in tickers:
      tickers = tickers[:-1] #Remove 'SPY' 
      weights = weights[:-1]
    for i, ticker in enumerate(tickers):
      ticker_vol = np.std(log_returns[ticker]) * np.sqrt(252)
      ticker_corr = log_returns[ticker].corr(port_returns_series)
      MRC = ticker_vol*ticker_corr
      PCR = (weights[i]*MRC)/port_vol
      PCRdict[ticker] = (f"{PCR*100:.2f}%")
    PCRframe = pd.DataFrame(data=PCRdict, index=["PCR"])

    metrics = pd.DataFrame(data=[[port_vol, sharpe, VaR_95, CVaR_95, mdd, beta]] ,columns=["Annual Volatilty", "Sharpe","95% VaR", "95% CVaR", "Max DD", "Beta"], index=["Portfolio"])
    return metrics, PCRframe

  except Exception as e:
    print(f"Error in calculate_metrics: {e}")
    return None


In [417]:
df = test.get_data('5y')
rand = monte_carlo(100,100,[0.353, 0.0, 0.4469, 0.2001], df, 'Medium', 'Tail risk', rand=True)
calculate_metrics([0.353, 0.0, 0.4469, 0.2001], rand)

(           Annual Volatilty    Sharpe   95% VaR  95% CVaR    Max DD     Beta
 Portfolio          0.470834 -2.919566 -0.047641 -0.076213 -0.527386  1.19959,
        AAPL   AMZN    GOOG    META
 PCR  32.16%  0.00%  46.77%  20.58%)

In [415]:
#test the simulated max and min sharpe metrics
df = test.get_data('10y')
mc = monte_carlo(504,50,[0.353, 0.0, 0.4469, 0.2001], df, regime="High", level='Moderate')
tickers = list(df.columns) + ['SPY']
sims = len(mc[tickers[0]][0])
all_metrics = []
all_PCR = []
for m in range(sims):
  mth_df = pd.DataFrame({ticker: mc[ticker][:,m] for ticker in tickers})
  metrics = calculate_metrics([0.353, 0.0, 0.4469, 0.2001], mth_df)
  all_metrics.append(metrics[0])
  all_PCR.append(metrics[1])
sharpes = [df.loc["Portfolio", "Sharpe"] for df in all_metrics]
min_idx = np.argmin(sharpes)
max_idx = np.argmax(sharpes)
all_metrics[min_idx].index = ["Worst Portfolio"]
all_metrics[max_idx].index = ["Best Portfolio"]
print(all_metrics[min_idx])
print(all_PCR[min_idx])
print(all_metrics[max_idx])
print(all_PCR[max_idx])

                 Annual Volatilty    Sharpe  95% VaR    Max DD      Beta
Worst Portfolio          0.579095 -1.586592 -0.06329 -0.887424  1.176545
       AAPL   AMZN    GOOG    META
PCR  31.98%  0.00%  44.12%  23.81%
                Annual Volatilty    Sharpe   95% VaR    Max DD      Beta
Best Portfolio           0.56331  2.447176 -0.053075 -0.367143  1.172476
       AAPL   AMZN    GOOG    META
PCR  33.08%  0.00%  42.89%  23.94%


In [ ]:
def historical(df:pd.DataFrame, crisis:str):
  """
  Returns the prices of your portfolio if during a historical crisis event.
  
  Parameters:
  - df: Dataframe of the historical adjusted close prices of the assets.
  - crisis: String of the event you want to simulate.

    Crisis Options:
    - "DOT-COM" -- The Dot-Com bubble
    - "2008 GFC" -- 2008 Global Financial Crisis
    - "2011 Euro" -- 2011 Eurozone Crisis
    - "COVID" -- COVID-19 Pandemic
    - "2022 Inf" -- 2022 Inflation Crash
  """
  try:
    crisis_periods = {"DOT-COM": ("2000-03-01", "2002-10-01"),
                      "2008 GFC": ("2007-10-01", "2009-03-01"),
                      "2011 Euro": ("2011-07-01", "2011-12-01"),
                      "COVID": ("2020-02-14", "2020-04-15"),
                      "2022 Inf": ("2022-01-01", "2022-10-01")
                      }
    crisis = crisis.strip()
    if crisis not in crisis_periods.keys():
      raise ValueError("Input a valid crisis event.")

    tickers = list(df.columns)
    start_date = pd.to_datetime(crisis_periods[crisis][0])
    end_date = pd.to_datetime(crisis_periods[crisis][1])

    if start_date not in df.index: #check if crisis event does not exist in existing df
      dfcrisis = yf.download(tickers, start=start_date, end=end_date, progress=False, auto_adjust=False)["Adj Close"]
    else:
      dfcrisis = df.loc[start_date:end_date]

    for ticker in tickers:
      if dfcrisis[ticker].isna().sum() >= len(dfcrisis[ticker])//3: #checks if any ticker reaches NA threshold
        raise ValueError(f"{ticker} price data does not exist for crisis period.")

    last_price = df.iloc[-1]
    crisisReturns = np.log(dfcrisis/dfcrisis.shift()).dropna()
    cumReturns = (1+crisisReturns).cumprod()
    crisisPrices = last_price.mul(cumReturns)
    return crisisPrices

  except Exception as e:
    print(f" \n Error in historical: {e}")
    return None

In [422]:
df = yf.download(["AAPL", "MSFT", "GOOG", "JNJ", "XOM"], period='10y', auto_adjust=False)["Adj Close"]
hist = historical(df, "COVID")
calculate_metrics([0.0, 0.056, 0.0, 0.444, 0.5], hist)

[*********************100%***********************]  5 of 5 completed


(           Annual Volatilty    Sharpe   95% VaR  95% CVaR    Max DD      Beta
 Portfolio          0.829948 -2.356408 -0.106502 -0.126618 -0.439156  1.097892,
       AAPL   GOOG    JNJ    MSFT     XOM
 PCR  0.00%  4.41%  0.00%  44.53%  49.81%)

In [ ]:
def factor_stress(df:pd.DataFrame, factor_dict:dict):
    """
    Computes stressed expected returns for each asset in the portfolio using a multi-factor model 
    and user-defined shocks to the factor premia.

    Parameters:
    - df: Dataframe of the historical adjusted close prices of the assets.
    - factor_dict: Dictionary of factor names and their respective shocks (e.g., {'SMB': 0.4, 'HML': -0.2}).
      = If 'FF3' or 'FF5' is entered as a key, either apply independent shocks via list or a single float for a constant shock as the value.

      factor options:
      - 'FF3' = Fama-French 3-Research Factors: ['Mkt-RF', 'SMB', 'HML']
      - 'FF5' = Fama-French 5-Research Factors: ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
      - 'Mkt-RF' = Market Returns - Risk-Free-Rate: Market Risk Premium.
      - 'SMB' = Small Minus Big: Returns of small-cap stocks minus large-cap stocks; measures the size anomaly - small-cap tend to outperform.
      - 'HML' = High Minus Low: Returns of high B/M (value) stocks minus low B/M (growth) stocks; measures the value anomaly - value stocks tend to outperform.
      - 'RMW' = Robust Minus Weak: Returns of firms with robust profability minus those with weak profitability; measures profitability factor - more robust firms tend to earn higher returns.
      - 'CMA' = Conservative Minus Aggressive: Returns of firms with conservative investment minus aggressive policies; captures investment factor - conservative tend to perform better.
      - 'MOM' = Momentum: Returns of stocks with high prior returns minus those with low prior returns; captures the momentum effect where past winners tend to continue outperforming in the short term.

      The user can input any combination of these factors that they desire.
    """
    try:
        if 'FF3' in factor_dict.keys():
            FF3_value = factor_dict.pop('FF3')
            if len(FF3_value) not in [1,3]:
                raise ValueError("Invalid shock length for 'FF3'. Either input a float for a uniform shock or a list of independent shocks for each factor.")

            if isinstance(FF3_value, list): #Unique shocks per factor in 'FF3'
                factor_dict.update({
                    'Mkt-RF': FF3_value[0],
                    'SMB': FF3_value[1],
                    'HML': FF3_value[2]
                    })
            else: #Uniform shock to all factors in 'FF3'
                factor_dict.update({
                    'Mkt-RF': FF3_value,
                    'SMB': FF3_value,
                    'HML': FF3_value
                    })
        elif 'FF5' in factor_dict.keys():
            FF5_value = factor_dict.pop('FF5')
            if isinstance(FF5_value, list): #Unique shocks per factor in 'FF5'
                factor_dict.update({
                    'Mkt-RF': FF5_value[0],
                    'SMB': FF5_value[1],
                    'HML': FF5_value[2],
                    'RMW': FF5_value[3],
                    'CMA': FF5_value[4]
                    })
            else: #Uniform shock to all factors in 'FF5'
                factor_dict.update({
                    'Mkt-RF': FF5_value,
                    'SMB': FF5_value,
                    'HML': FF5_value,
                    'RMW': FF5_value,
                    'CMA': FF5_value
                    })
            
        factors = list(factor_dict.keys())
        shocks = list(factor_dict.values())

        #Read and clean factor csvs
        FFdf = pd.read_csv('../data/F-F_Research_Data_5_Factors_2x3_daily.csv', skiprows=3, index_col=0)
        FFdf = FFdf.iloc[:-1]
        FFdf.index = pd.to_datetime(FFdf.index)
        MOMdf = pd.read_csv('../data/F-F_Momentum_Factor_daily.csv', skiprows=13, index_col=0)
        MOMdf = MOMdf.rename(columns={'Mom':'MOM'})
        MOMdf = MOMdf.iloc[:-1]
        MOMdf.index = pd.to_datetime(MOMdf.index)

        #Align both factor csvs and only grab necessary factors
        factor_align = MOMdf.index.intersection(FFdf.index)
        FFdf = FFdf.loc[factor_align]
        MOMdf = MOMdf.loc[factor_align]
        factors_df = pd.concat([FFdf, MOMdf], axis=1)
        rf = factors_df['RF'].copy()
        factors_df = factors_df.loc[:, factors]
        
        #Align historical asset returns with joint factor df
        log_returns = np.log(df/df.shift()).dropna()
        aligned_index = log_returns.index.intersection(factors_df.index) 
        log_returns = log_returns.loc[aligned_index]
        factors_df = factors_df.loc[aligned_index]/100
        factor_means = factors_df.mean()
        rf = rf.loc[aligned_index]/100

        #Fit OLS on each ticker and store neccesary values
        results = {}
        stressed_means = {}
        tickers = list(df.columns)
        for ticker in tickers:
            excess_returns = log_returns[ticker] - rf
            X = sm.add_constant(factors_df)
            y = excess_returns

            model = sm.OLS(y, X).fit()
            results[ticker] = {
                'alpha': model.params['const'],
                'betas': model.params.drop('const').to_dict(),
                'r-squared': model.rsquared
            }
            #Compute stressed mean returns
            alpha = results[ticker]['alpha']
            betas = results[ticker]['betas']
            stressed_mean = alpha
            for factor in factors:
                stressed_mean += betas[factor]*factor_means[factor]*(1+factor_dict[factor])
            stressed_means[ticker] = stressed_mean
        stressed_means['SPY'] = 0.0
        stressed_means_list = list(map(float, list(stressed_means.values())))

        return stressed_means_list

    except Exception as e:
        print(f"Error in factor_stress: {e}")
        return None

In [302]:
df = test.get_data('5y')
factors = factor_stress(df, {'FF3': [0, 0.2, 0.1], 'MOM': -0.4})
sim = monte_carlo(T=25, sims=5, weights=[0.0, 0.0596, 0.0, 0.4404, 0.5], df=df, regime="Low", level='Regulatory', factorReturns=factors, rand=True)
calculate_metrics([0.0, 0.0596, 0.0, 0.4404, 0.5], sim)

(           Annual Volatilty    Sharpe  95% VaR  95% CVaR    Max DD      Beta
 Portfolio           0.45551 -3.994335 -0.05634 -0.059835 -0.261543  1.218546,
       AAPL   GOOG    JNJ    MSFT     XOM
 PCR  0.00%  4.41%  0.00%  37.14%  56.43%)